#### worth notice
https://medium.com/@mikkokotila/a-comprehensive-list-of-hyperparameter-optimization-tuning-solutions-88e067f19d9

In [10]:
%load_ext autoreload
%autoreload 2

In [11]:
import os
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [12]:
from utils import get_data, R2_1, round_params

In [13]:
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

# Get data

In [14]:
DATA_PATH = "data"
train = pd.read_csv(os.path.join(DATA_PATH, 'trainingData.csv'))
valid = pd.read_csv(os.path.join(DATA_PATH, 'validationData.csv'))

In [15]:
target = "winRate"
col_names = ["nofGames", "nOfPlayers", "winRate"]
train, valid = get_data(train, valid)

In [16]:
x_train = train.drop(col_names, axis=1).values
y_train = train[target].values

x_val = valid.drop(col_names, axis=1).values
y_val = valid[target].values

# Hyperopt

In [ ]:
# in case of need
! pip install hyperopt

In [ ]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import stochastic

In [ ]:
def svm_objective(params):
  
    hp_params_svm = {**params, **base_params}
    svr_regressor = SVR(**params)    
    svr_regressor.fit(x_train[: CUTOFF], y_train[: CUTOFF])

    pred = svr_regressor.predict(x_val)
    mse = mean_squared_error(y_val, pred)

    return{'loss': mse, 'status': STATUS_OK }

In [ ]:
hp_params = {
    "gamma": hp.uniform("gamma", 10**-3, 10**1),
    "epsilon": hp.uniform("epsilon", 10**-3, 10**0),
    "C": hp.uniform("C", 10**0, 10**3)   
}
# example -> operationalization of stochastic expression
stochastic.sample(hp_params)

In [ ]:
trials = Trials()
best = fmin(
    fn = svm_objective,
    space = hp_params,
    algo = tpe.suggest,
    max_evals = 100,
    trials = trials
)
# random seed ?

In [ ]:
best = round_params(best)
best_params_hyperopt = {**best, **base_params}
best_params_hyperopt["verbose"] = 1
best_params_hyperopt

In [ ]:
hyperopt_results = pd.DataFrame({'loss': [x['loss'] for x in trials.results], 'iteration': np.arange(len(trials.results)),
                            **trials.idxs_vals[1]})
hyperopt_results.plot(kind="line", x="iteration", y=param_name, layout=(1,3),
                 subplots = True, sharex = False, marker="o", ls="none", figsize=(25, 5));
hyperopt_results.plot(kind="kde", y=param_name, layout=(1,3),
                 subplots = True, sharex = False, figsize=(25, 5));

#### Hyperopt summary

In [ ]:
svr_regressor = SVR(**best_params_hyperopt)    
svr_regressor.fit(x_train[: CUTOFF], y_train[: CUTOFF])
svr_regressor.score(x_val, y_val)

In [ ]:
pred = svr_regressor.predict(x_val)
mean_squared_error(y_val, pred)

In [ ]:
R2_1(y_val, pred)

# Hyperopt - sklearn

In [ ]:
# in case of need
! git clone git@github.com:hyperopt/hyperopt-sklearn.git & (cd hyperopt-sklearn && pip install -e .)

In [ ]:
from hpsklearn import HyperoptEstimator, svr_rbf
from hyperopt import tpe

In [ ]:
# sprawdz inne algorytmy optymalizacji bo jak na razie z samym tpe.suggest cieniutko
estim = HyperoptEstimator(
    regressor=svr_rbf('my_clf', shrinking=False),
    algo=tpe.suggest,
    max_evals=100,
    trial_timeout=300,
    loss_fn = mean_squared_error
)

In [ ]:
estim.fit(x_train[: CUTOFF], y_train[: CUTOFF])
estim.score(x_val, y_val)

In [ ]:
best_params_hyperopt_sk = estim.trials.argmin
val_params = [best_params_hyperopt_sk['my_clf.rbf_C'], best_params_hyperopt_sk['my_clf.rbf_epsilon'], best_params_hyperopt_sk['my_clf.rbf_gamma']]
val_params

In [ ]:
best_params_hyperopt_sk = upgrade_params(base_params, value_base_params=val_params, verbose=1)
best_params_hyperopt_sk

In [ ]:
svr_regressor = SVR(**best_params_skopt)    
svr_regressor.fit(x_train[: CUTOFF], y_train[: CUTOFF])
svr_regressor.score(x_val, y_val)

# hypopt

In [ ]:
# in case of need
! pip install hypopt

In [ ]:
hp_params = {
    "gamma": hp.uniform("gamma", 0.01, 10),
    "epsilon": hp.uniform("epsilon", 0.005, 1),
    "C": hp.uniform("C", 1, 100)   
}
hp_params_svm = {**hp_params, **base_params}
def svm_objective(params):

    svr_regressor = SVR(**params)    
    svr_regressor.fit(x_train[: CUTOFF], y_train[: CUTOFF])


    pred = svr_regressor.predict(x_val)
    mse = mean_squared_error(y_val, pred)
    mae = mean_absolute_error(y_val, pred)
    r2 = R2_1(y_val, pred)
    svr_score = svr_regressor.score(x_val, y_val)
    print("mse: {:.6f}, mae: {:.6f}, r2: {:.6f}, svr_score(r2): {:.6f} \n".format(mse, mae, r2, svr_score))

    return{'loss': mse, 'status': STATUS_OK }

stochastic.sample(hp_params_svm)

# SKOPT

In [ ]:
from skopt import gp_minimize, forest_minimize, gbrt_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

In [ ]:
def upgrade_params(base_params, value_base_params, verbose=1):
    best_params = round_params(dict(zip(param_name, value_base_params)))
    best_params = {**base_params, **best_params}
    best_params["verbose"] = verbose
    return best_params

def show_optimization_process():
    pass

In [ ]:
space  = [
    Real(10**-2, 10**1, "uniform", name='gamma'),
    Real(10**-3, 10**0, "uniform", name='epsilon'),
    Real(10**0, 10**2, "uniform", name='C')
]

In [ ]:
CUTOFF = 1000
base_params = {
    "kernel": "rbf",
    "shrinking": False
}
@use_named_args(space)
def svm_objective(**params):
    params = {**params, **base_params}
    svr_regressor = SVR(**params)    
    svr_regressor.fit(x_train[: CUTOFF], y_train[: CUTOFF])
    pred = svr_regressor.predict(x_val)
    return mean_squared_error(y_val, pred)


## Gaussian Process

In [ ]:
res_gp = gp_minimize(
    func=svm_objective,
    dimensions=space,
    acq_func="EI",
    n_calls=100,
    n_jobs=4,
    random_state=42
)
res_gp.fun

In [ ]:
best_params_skopt = upgrade_params(base_params, value_base_params=res_gp.x, verbose=1)
best_params_skopt

In [ ]:
skopt_results = pd.DataFrame(data=res_gp.x_iters, columns=param_name)
skopt_results["iteration"] = np.arange(skopt_results.shape[0])
skopt_results.plot(kind="line", x="iteration", y=param_name, layout=(1,3),
                 subplots = True, sharex = False, marker="o", ls="none", figsize=(25, 5));
skopt_results.plot(kind="kde", y=param_name, layout=(1,3),
                 subplots = True, sharex = False, figsize=(25, 5));

#### Skopt summary

In [ ]:
svr_regressor = SVR(**best_params_skopt)    
svr_regressor.fit(x_train[: CUTOFF], y_train[: CUTOFF])
svr_regressor.score(x_val, y_val)

In [ ]:
pred = svr_regressor.predict(x_val)
R2_1(y_val, pred)

### GBR Trees Optimizer

In [ ]:
res_gbr = gbrt_minimize(
    func=svm_objective,
    dimensions=space,
    acq_func="EI",
    n_calls=100,
    n_jobs=4,
    random_state=42
)
res_gbr.fun

In [ ]:
best_params_skopt = dict(zip(param_name, res_gbr.x))
best_params_skopt = round_params(best_params_skopt)
best_params_skopt = {**base_params, **best_params_skopt}
best_params_skopt["verbose"] = 1
best_params_skopt

In [ ]:
svr_regressor = SVR(**best_params_skopt)    
svr_regressor.fit(x_train[: CUTOFF], y_train[: CUTOFF])
svr_regressor.score(x_val, y_val)

In [ ]:
pred = svr_regressor.predict(x_val)
R2_1(y_val, pred)

### Tree-Based Optimizer

In [ ]:
res_forest = forest_minimize(
    func=svm_objective,
    dimensions=space,
    acq_func="EI",
    n_calls=100,
    n_jobs=4,
    random_state=42
)
res_forest.fun

In [ ]:
best_params_skopt = dict(zip(param_name, res_forest.x))
best_params_skopt = round_params(best_params_skopt)
best_params_skopt = {**base_params, **best_params_skopt}
best_params_skopt["verbose"] = 1
best_params_skopt

In [ ]:
svr_regressor = SVR(**best_params_skopt)    
svr_regressor.fit(x_train[: CUTOFF], y_train[: CUTOFF])
svr_regressor.score(x_val, y_val)

In [ ]:
pred = svr_regressor.predict(x_val)
R2_1(y_val, pred)

## Bayes search

In [ ]:
# in case of need
!pip install bayesian-optimization

In [ ]:
from skopt import BayesSearchCV

In [ ]:
opt = BayesSearchCV(
    estimator=SVR(kernel="rbf", shrinking=False),
    search_spaces={
        'C': (10**0, 10**2, 'uniform'),  
        'gamma': (10**-3, 10**0, 'uniform'),
        'epsilon': (10**-2, 10**1, 'uniform')
    },
    n_iter=100,
    n_jobs=4,
    cv=4,
    random_state=42
)

opt.fit(x_train[: CUTOFF], y_train[: CUTOFF])
opt.score(x_val, y_val)

In [ ]:
pred = opt.predict(x_val)
mean_squared_error(y_val, pred)

In [ ]:
R2_1(y_val, pred)

# BayesianOptimization 

In [ ]:
# in case of need
! pip install bayesian-optimization

In [22]:
from bayes_opt import BayesianOptimization
from sklearn.svm import SVR

def svr_funct(C, gamma, epsilon):
    svr_regressor = SVR(kernel="rbf", shrinking=False, C=C, gamma=gamma, epsilon=epsilon)    
    svr_regressor.fit(x_train[: CUTOFF], y_train[: CUTOFF])
    pred = svr_regressor.predict(x_val)
    return - mean_squared_error(y_val, pred)


# Bounded region of parameter space
pbounds = {
    'C': (10**0, 10**3),  
    'gamma': (10**-3, 10**1),
    'epsilon': (10**-3, 10**0)
}

optimizer = BayesianOptimization(
    f=svr_funct,
    pbounds=pbounds,
    random_state=42
)

In [23]:
optimizer.maximize()

|   iter    |  target   |     C     |  epsilon  |   gamma   |
-------------------------------------------------------------
|  1        | -0.05512  |  375.2    |  0.9508   |  7.32     |
|  2        | -0.02096  |  599.1    |  0.1569   |  1.561    |
|  3        | -0.05512  |  59.03    |  0.8663   |  6.012    |
|  4        | -0.002402 |  708.4    |  0.02156  |  9.699    |
|  5        | -0.03808  |  832.6    |  0.2131   |  1.819    |
|  6        | -0.05512  |  647.9    |  1.0      |  10.0     |
|  7        | -0.002162 |  1e+03    |  0.001    |  0.001    |
|  8        | -0.0021   |  202.2    |  0.001    |  0.001    |
|  9        | -0.001994 |  1.0      |  0.001    |  0.001    |
|  10       | -0.002109 |  487.5    |  0.001    |  0.001    |
|  11       | -0.05512  |  737.7    |  1.0      |  0.001    |
|  12       | -0.002205 |  929.5    |  0.001    |  10.0     |
|  13       | -0.002205 |  274.6    |  0.001    |  10.0     |
|  14       | -0.002205 |  541.2    |  0.001    |  10.0     |
|  15   